In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# CIFAR10 is a set of 60K images of 3 color channels of size 32x32.
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

In [4]:
# OTHER CONSTANTS
BATCH_SIZE = 32
EPOCHS = 20
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.RMSprop()

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [6]:
# Image augmnetation
datagen = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range= 0.2,
    height_shift_range = 0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

In [7]:
# Defining the model.
def build(input_shape, classes):
    model = models.Sequential()
    
    # 1st block of the model.
    # conv -> batchNormalization -> conv -> batchNormalization -> maxPooling -> dropout.
    model.add(layers.Convolution2D(32, (3,3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))
        
    # 2nd block of the model.
    # conv -> batchNormalization -> conv -> batchNormalization -> maxPooling -> dropout.
    model.add(layers.Convolution2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))
    
    # 3rd block of the model.
    # conv -> batchNormalization -> conv -> batchNormalization -> maxPooling -> dropout.
    model.add(layers.Convolution2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))

    # Dense
    model.add(layers.Flatten())
    model.add(layers.Dense(classes, activation ='softmax'))
    return model
    

In [8]:
# Callbacks.
callbacks = [
    # Write tensorboard logs to ./logs directory.
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [9]:
X_train = X_train.reshape(-1,32,32,3)
X_test = X_test.reshape(-1,32,32,3)
X_train = X_train/255.0
X_test = X_test/255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=CLASSES)
y_train.shape

(50000, 10)

In [ ]:
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)
model = build(INPUT_SHAPE, CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
history = model.fit(X_train,y_train,batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, verbose=VERBOSE, callbacks=callbacks)

Epoch 1/20
   1/1250 [..............................] - ETA: 1s - loss: 4.7613 - accuracy: 0.0625WARNING:tensorflow:From c:\users\archi\scoop\apps\miniconda3\current\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1250/1250 [==============================] - 1014s 811ms/step - loss: 1.8254 - accuracy: 0.4509 - val_loss: 1.4937 - val_accuracy: 0.5807
Epoch 2/20
1250/1250 [==============================] - 1261s 1s/step - loss: 1.1267 - accuracy: 0.6308 - val_loss: 1.0308 - val_accuracy: 0.6395
Epoch 3/20
1250/1250 [==============================] - 1361s 1s/step - loss: 0.8911 - accuracy: 0.6996 - val_loss: 1.0294 - val_accuracy: 0.6541
Epoch 4/20
1250/1250 [==============================] - 768s 614ms/step - loss: 0.7638 - accuracy: 0.7391 - val_loss: 0.7219 - val_accuracy: 0.7497
Epoch 5/20
1250/1250 [=

In [ ]:
score = model.evaluate(X_test, y_test, batch_size= BATCH_SIZE, verbose = VERBOSE)
print("\nTest Score: ", score[0])
print("Test Accuracy:", score[1])

In [ ]:
log_dir = './logs'
%tensorboard --logdir log_dir